In [1]:
file_path = (
    "Resume_11_2024.pdf"
)

In [2]:
from PyPDF2 import PdfReader

In [33]:
reader = PdfReader(file_path)
print(len(reader.pages))
text = ""
for page in reader.pages:
    text += page.extract_text()
# page = reader.pages[0]
# text = page.extract_text()


2


In [34]:
print(text)

Seyhan Emre Gorucu
Calgary, AB, Canada
(403) 827-4222 | segorucu@gmail.com ||| Canadian citizen & US Greencard holder
WorkExperience
ComputerModellingGroup Calgary,AB
SCIENTIFIC SOFTWARE DEVELOPMENT ENGINEER / SENIOR SIMULATION SCIENTIST Aug2013-Present
•Workedondevelopment,maintenanceandsupportofGEM(Fortranbasedequation-of-statecompositionalsimulatorusedforthesimulation
ofoil-gas-waterflow).
•Developedand/orimprovedfeaturesforsimulationofaqueousreactions,mineralreactions,surfacecomplexation,pitzeractivitymodel,phase
behavior,finesandproppants,single-componentisenthalpicflashcalculations.
•Thesefeaturesareusedbyhundredsofenergycompaniesinordertoimprovedecisionmakingforreservoirmanagement.
•ConsultingandSupport.
ThePennsylvaniaStateUniversity StateCollege,PA
RESEARCH ASSISTANT Aug. 2010-Apr. 2013
•Introduced new algorithms that compute the number of physical phases and chemical compositions in an oil and gas reservoir. As these
calculationsarerepeatedbillionsoftimesduringanumericalr

In [3]:
OPENAI_API_KEY = # your api key

In [4]:
import openai
import os
from openai import OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
prompt = f"""Given the following ocr result from a resume, put it in a readable format.
    OCR result
    {text}
"""

NameError: name 'text' is not defined

In [38]:
chat_response = openai_client.chat.completions.create(
        model="gpt-4o",  # Specify the GPT-4 model
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )

In [39]:
resume_clean = chat_response.choices[0].message.content

In [40]:
print(resume_clean)

Here is a formatted and readable version of the provided OCR resume data:

---

### Seyhan Emre Gorucu  
Calgary, AB, Canada  
Phone: (403) 827-4222 | Email: segorucu@gmail.com  
LinkedIn: [Profile URL] | GitHub: [Profile URL]  
**Canadian citizen & US Greencard holder**

---

### Work Experience

#### Computer Modelling Group, Calgary, AB  
**Scientific Software Development Engineer / Senior Simulation Scientist**  
_August 2013 - Present_  
- Worked on the development, maintenance, and support of GEM (Fortran-based equation-of-state compositional simulator for oil-gas-water flow simulation).
- Developed and improved features for the simulation of aqueous reactions, mineral reactions, surface complexation, Pitzer activity model, phase behavior, fines and proppants, single-component isenthalpic flash calculations.
- These features are used by hundreds of energy companies to improve decision-making for reservoir management.
- Provided consulting and support.

#### The Pennsylvania State

In [6]:
from elasticsearch import Elasticsearch

In [7]:
es_client = Elasticsearch(
        ['https://localhost:9200'],
        basic_auth=('elastic', 'elastic'),
        verify_certs = False
    )

C:\Users\segor\miniconda3\envs\brave\Lib\site-packages\elasticsearch\_sync\client\__init__.py:402: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [43]:
def get_matching_jobs(index_name, location, position):
    query = {
        "query": {
            "bool": {
                "must": [
                    {"match": {"location": location}},
                    {"match": {"query": position}}
                ]
            }
        }
    }
    results = es_client.search(index=index_name, body=query, size=1000)
    return results['hits']['hits']


In [55]:
jobs = get_matching_jobs("brave_project","Toronto", "Product Manager")

C:\Users\segor\AppData\Local\Temp\ipykernel_9200\1041610920.py:12: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  results = es_client.search(index=index_name, body=query, size=1000)
C:\Users\segor\miniconda3\envs\brave\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [56]:
print(len(jobs))

10


In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
def load_split_pdf(file_path):
    # Load the PDF document and split it into chunks
    loader = PyPDFLoader(file_path)  # Initialize the PDF loader with the file path
    documents = loader.load()  # Load the PDF document 

    # Initialize the recursive character text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,  # Set the maximum chunk size
        chunk_overlap=20,  # Set the number of overlapping characters between chunks
        separators=["\n\n", "\n", " ", ""],  # Define resume-specific separators for splitting
    )   

    # Split the loaded documents into chunks
    chunks = text_splitter.split_documents(documents)
    return documents, chunks

In [13]:
resume_docs, resume_chunks = load_split_pdf(file_path)

In [16]:
full_resume = " ".join([doc.page_content for doc in resume_docs])

In [18]:
prompt = f"""Summarize the following resume.
    {full_resume}
"""
chat_response = openai_client.chat.completions.create(
        model="gpt-4o",  # Specify the GPT-4 model
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )

In [19]:
resume_summary = chat_response.choices[0].message.content

In [20]:
print(resume_summary)

**Resume Summary: Seyhan Emre Gorucu**

- **Location & Contact**: Based in Calgary, AB, Canada. Canadian citizen and US Greencard holder. Contact via segorucu@gmail.com or phone: (403) 827-4222.

- **Current Position**: Scientific Software Development Engineer / Senior Simulation Scientist at Computer Modelling Group in Calgary, AB since August 2013. Responsible for the development and maintenance of the GEM simulator for oil-gas-water flow, improving simulation features for various chemical reactions and phase behavior, used by energy companies for reservoir management.

- **Research Experience**: Previously worked as a Research Assistant at The Pennsylvania State University, introducing faster and more robust algorithms for oil and gas reservoir simulations. Internship experience at Quantum Reservoir Impact developing an EOR Screening and Forecasting Toolbox.

- **Teaching**: Served as a Teaching Assistant at The Pennsylvania State University from January 2009 to December 2011.

- **

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Initialize the Hugging Face embedding model


def create_vector_store(chunks):
    # Store embeddings into the vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = FAISS.from_documents(
        documents=chunks,  # Input chunks to the vector store
        embedding=embeddings  # Use the initialized embeddings model
    )
    return vector_store

In [22]:
vector_store = create_vector_store(resume_chunks)

In [52]:
print(vector_store)

In [23]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

In [50]:
load_dotenv()  # Load environment variables from .env file
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model_name="mixtral-8x7b-32768")

In [51]:
def analyze_resume(full_resume, job_description):
    
    # Template for analyzing the resume against the job description
    template = """
    You are an AI assistant specialized in resume analysis and recruitment. Analyze the given resume and compare it with the job description. 
    Provide only an integer representing the match percentage between 0 and 100. Do not include any additional text or explanations.

    Resume: {resume}
    Job Description: {job_description}
    """
    
    prompt = PromptTemplate(  # Create a prompt template with input variables
        input_variables=["resume", "job_description"],
        template=template
    )

    # Create a chain combining the prompt and the language model
    chain = prompt | llm

    # Invoke the chain with input data
    response = chain.invoke({"resume": full_resume, "job_description": job_description})

    # Return the content of the response
    return response.content


In [70]:
ranking = []
for i,job in enumerate(jobs):
    content = analyze_resume(resume_summary, job["_source"]["summary"])
    for j in range(min(3,len(content))):
        if not content[j].isdigit():
            break
    else:
        j = 3
    if j == 0:
        continue
    rate = int(content[0:j])
    ranking.append((rate,i))
    print(rate,content[0:3])

30 30 
15 15 
10 10

10 10

10 10

10 10

20 20

10 10

10 10

60 60



In [73]:
ranking.sort(reverse=True)
ranking

[(60, 9),
 (30, 0),
 (20, 6),
 (15, 1),
 (10, 8),
 (10, 7),
 (10, 5),
 (10, 4),
 (10, 3),
 (10, 2)]